In [55]:
import tensorflow as tf
from gene_expression import *
from pathway_hierarchy import *
from utils import *
import torch

In [56]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [57]:
import pandas as pd
import numpy as np

In [58]:
tf.test.is_gpu_available()

2024-07-10 08:45:35.656627: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-10 08:45:35.656800: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-10 08:45:35.656888: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

True

In [59]:
torch.cuda.is_available()


True

In [60]:
rdata = pd.read_csv('../../preprocessed_data/inhibitory_neuron/train_test_set/train_count_matrix.csv', index_col=0)
qdata = pd.read_csv('../../preprocessed_data/inhibitory_neuron/train_test_set/test_count_matrix.csv', index_col=0)
rlabel = pd.read_csv('../../preprocessed_data/inhibitory_neuron/train_test_set/train_label.csv')
qlabel = pd.read_csv('../../preprocessed_data/inhibitory_neuron/train_test_set/test_label.csv')

In [61]:
single_cell_metadata = pd.read_csv('../../preprocessed_data/inhibitory_neuron/metadata_inhibitory_neurons.csv')

In [62]:
single_cell_metadata

,cell_id,orig.ident,nCount_RNA,nFeature_RNA,percent.mt,percent.rb,log10GenesPerUMI,cell_type_high_resolution,subject,Pathologic_diagnosis_of_AD,individualID,clinical_diagnosis,pathological_diagnosis,clinical_pathological_AD,train_test_clinical_and_pathological
0,AAACCCAAGAAATCCA.12.9,ROSMAP,13490,4276,0.237213,0.289103,0.879183,Inh L3-5 SST MAFB,ROSMAP-65967,yes,R3857147,AD,AD,AD_with_Plaques,train
1,AAACCCAAGAACGCGT.6.6,ROSMAP,2651,1533,8.185590,0.452659,0.930517,Inh L3-5 SST MAFB,ROSMAP-90639,yes,R7090624,NCI,AD,False,NaN
2,AAACCCAAGAACTGAT.10.12,ROSMAP,6550,2764,4.809160,0.274809,0.901814,Inh LAMP5 RELN,ROSMAP-57958,yes,R2347173,NCI,AD,False,NaN
3,AAACCCAAGAAGCGGG.31.8,ROSMAP,11658,4377,5.738549,0.394579,0.895381,Inh VIP CLSTN2,ROSMAP-40761,yes,R1287407,False,AD,False,NaN
4,AAACCCAAGAATCCCT.14.8,ROSMAP,15297,4688,0.921749,0.307250,0.877260,Inh VIP CLSTN2,ROSMAP-68841,yes,R3757880,NCI,AD,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329694,TTTGTTGTCTTAGCAG.7.9,ROSMAP,9309,3459,2.062520,0.225588,0.891670,Inh VIP TSHZ2,ROSMAP-77886,no,R2554598,NCI,False,False,NaN
329695,TTTGTTGTCTTCCCGA.28.6,ROSMAP,22844,5507,0.153213,0.358956,0.858250,Inh PVALB SULF1,ROSMAP-38931,no,R6292415,NCI,No AD,NCI_with_No_Plaques,train
329696,TTTGTTGTCTTCGACC.3.13,ROSMAP,16770,5431,2.098986,0.679785,0.884093,Inh CUX2 MSR1,ROSMAP-53472,yes,R3863249,False,False,False,NaN
329697,TTTGTTGTCTTGGTGA.8.11,ROSMAP,9475,3447,0.147757,0.179420,0.889569,Inh VIP ABI3BP,ROSMAP-44788,no,R5221394,NCI,No AD,NCI_with_No_Plaques,test


In [63]:
# Verifying Train Label is correct

list(single_cell_metadata.set_index('cell_id').loc[rdata.columns,].clinical_pathological_AD) == list(rlabel.diagnosis)

True

In [64]:
# Verifying Test Label is correct

list(single_cell_metadata.set_index('cell_id').loc[qdata.columns,].clinical_pathological_AD) == list(qlabel.diagnosis)

True

In [65]:
rdata = rdata.T
qdata = qdata.T
rdata

ensemble_gene_name,ENSG00000163463,ENSG00000163462,ENSG00000185499,ENSG00000169231,ENSG00000261905,ENSG00000263290,ENSG00000261893,ENSG00000263324,ENSG00000262785,ENSG00000160752,...,ENSG00000228253,ENSG00000198899,ENSG00000198938,ENSG00000198840,ENSG00000212907,ENSG00000198886,ENSG00000198786,ENSG00000198695,ENSG00000198727,ENSG00000274847
AAACCCAAGAAATCCA.12.9,0,0,0,0,0,0,0,2,0,1,...,0,3,2,1,0,0,3,0,4,0
AAACCCAAGATTCGCT.20.4,0,0,0,0,0,0,0,1,0,1,...,0,40,34,6,0,12,4,0,14,0
AAACCCAAGCCATCCG.21.6,0,0,0,0,0,0,0,0,0,0,...,0,4,5,1,0,3,0,0,1,0
AAACCCAAGCCTATTG.18.6,0,0,0,0,0,0,0,0,0,0,...,0,23,40,11,2,21,2,0,18,0
AAACCCAAGCTAGATA.25.10,0,1,0,0,0,1,0,0,0,0,...,1,90,118,43,0,32,10,0,60,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGTCACACCGG.11.9,0,0,0,0,0,0,0,0,0,0,...,1,27,47,11,1,8,6,0,19,0
TTTGTTGTCCGAAATC.9.14,0,0,0,0,0,0,0,0,0,1,...,0,4,4,0,0,4,0,0,3,0
TTTGTTGTCGAACCTA.5.2,0,0,0,0,0,0,0,0,0,0,...,0,15,33,3,0,6,2,0,5,0
TTTGTTGTCTTACACT.27.3,0,0,0,0,0,0,0,0,0,1,...,1,21,36,9,1,7,4,0,25,0


In [66]:
rdata['diagnosis'] =  list(rlabel.diagnosis)
qdata['diagnosis'] =  list(qlabel.diagnosis)

In [67]:
rdata.diagnosis.value_counts()

diagnosis
NCI_with_No_Plaques    29147
AD_with_Plaques        24372
Name: count, dtype: int64

In [68]:
rdata_neg = rdata[rdata.diagnosis == 'NCI_with_No_Plaques'].iloc[0:24372,]
rdata_pos = rdata[rdata.diagnosis == 'AD_with_Plaques'].iloc[0:24372,]
rdata = pd.concat([rdata_neg,rdata_pos])


In [69]:
rdata

ensemble_gene_name,ENSG00000163463,ENSG00000163462,ENSG00000185499,ENSG00000169231,ENSG00000261905,ENSG00000263290,ENSG00000261893,ENSG00000263324,ENSG00000262785,ENSG00000160752,...,ENSG00000198899,ENSG00000198938,ENSG00000198840,ENSG00000212907,ENSG00000198886,ENSG00000198786,ENSG00000198695,ENSG00000198727,ENSG00000274847,diagnosis
AAACCCAAGATTCGCT.20.4,0,0,0,0,0,0,0,1,0,1,...,40,34,6,0,12,4,0,14,0,NCI_with_No_Plaques
AAACCCAAGCTAGTTC.31.9,0,0,0,0,0,0,0,0,0,2,...,40,67,10,0,35,2,0,25,0,NCI_with_No_Plaques
AAACCCAAGGTGCTAG.19.10,0,0,0,0,0,0,0,0,0,0,...,7,8,4,0,1,0,0,3,0,NCI_with_No_Plaques
AAACCCACAACAAGAT.11.11,0,0,0,0,0,0,0,0,0,0,...,13,13,4,0,8,0,0,3,0,NCI_with_No_Plaques
AAACCCACAAGTGGTG.5.2,0,0,0,0,0,1,0,0,0,1,...,29,40,5,0,25,5,0,15,0,NCI_with_No_Plaques
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGGTGACACAG.4.13,0,0,0,0,1,1,0,1,0,0,...,6,26,6,1,6,0,0,8,0,AD_with_Plaques
TTTGTTGGTGAGCTCC.13.12,0,0,0,0,0,0,0,0,0,0,...,19,26,3,0,18,1,0,7,0,AD_with_Plaques
TTTGTTGGTGGATCGA.32.3,0,1,0,0,0,0,0,0,0,0,...,14,4,17,1,3,2,0,1,0,AD_with_Plaques
TTTGTTGTCACACCGG.11.9,0,0,0,0,0,0,0,0,0,0,...,27,47,11,1,8,6,0,19,0,AD_with_Plaques


In [70]:
rdata = rdata.sample(frac = 1)

In [71]:
rdata

ensemble_gene_name,ENSG00000163463,ENSG00000163462,ENSG00000185499,ENSG00000169231,ENSG00000261905,ENSG00000263290,ENSG00000261893,ENSG00000263324,ENSG00000262785,ENSG00000160752,...,ENSG00000198899,ENSG00000198938,ENSG00000198840,ENSG00000212907,ENSG00000198886,ENSG00000198786,ENSG00000198695,ENSG00000198727,ENSG00000274847,diagnosis
GTCTGTCTCAACTGGT.4.13,0,0,0,0,0,0,0,0,0,1,...,10,7,4,1,9,3,0,11,0,AD_with_Plaques
CACAAACCAGCTGGCT.19.1,0,0,0,0,0,0,3,0,0,0,...,7,21,4,0,11,0,0,8,0,AD_with_Plaques
TCCGTGTTCGGTTGTA.27.9,0,0,0,1,0,0,1,0,0,0,...,27,48,17,2,29,9,1,36,0,AD_with_Plaques
CACTCCACATGTCGAT.28.0,0,0,0,0,0,0,0,0,0,0,...,9,10,4,0,7,0,0,3,0,AD_with_Plaques
ATTCTTGCACCCATAA.7.6,0,0,0,0,1,0,0,0,0,2,...,18,21,11,0,11,2,0,7,0,NCI_with_No_Plaques
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GGTGTTATCAACTCTT.31.3,0,0,0,0,1,0,0,0,0,1,...,84,94,45,2,54,4,0,56,0,AD_with_Plaques
TAGAGTCTCCCATTTA.4.13,0,0,0,0,0,0,0,0,0,0,...,12,21,12,0,16,2,0,4,0,AD_with_Plaques
GAGTTGTTCTTCTTCC.23.10,0,0,0,0,0,0,1,0,0,0,...,63,65,23,0,20,11,0,41,0,AD_with_Plaques
AACAAAGGTCGAGTTT.31.9,0,0,0,0,0,0,0,0,0,0,...,48,59,4,0,20,3,1,20,0,NCI_with_No_Plaques


In [72]:
rlabel = pd.DataFrame(rdata.diagnosis)

In [73]:
rlabel

,diagnosis
GTCTGTCTCAACTGGT.4.13,AD_with_Plaques
CACAAACCAGCTGGCT.19.1,AD_with_Plaques
TCCGTGTTCGGTTGTA.27.9,AD_with_Plaques
CACTCCACATGTCGAT.28.0,AD_with_Plaques
ATTCTTGCACCCATAA.7.6,NCI_with_No_Plaques
...,...
GGTGTTATCAACTCTT.31.3,AD_with_Plaques
TAGAGTCTCCCATTTA.4.13,AD_with_Plaques
GAGTTGTTCTTCTTCC.23.10,AD_with_Plaques
AACAAAGGTCGAGTTT.31.9,NCI_with_No_Plaques


In [74]:
rdata = rdata.drop(columns=['diagnosis'])

In [75]:
qdata.diagnosis.value_counts()

diagnosis
NCI_with_No_Plaques    5241
AD_with_Plaques        3442
Name: count, dtype: int64

In [76]:
qdata_neg = qdata[qdata.diagnosis == 'NCI_with_No_Plaques'].iloc[0:3442,]
qdata_pos =qdata[qdata.diagnosis == 'AD_with_Plaques'].iloc[0:3442,]
qdata = pd.concat([qdata_neg,qdata_pos])

In [77]:
qdata

ensemble_gene_name,ENSG00000163463,ENSG00000163462,ENSG00000185499,ENSG00000169231,ENSG00000261905,ENSG00000263290,ENSG00000261893,ENSG00000263324,ENSG00000262785,ENSG00000160752,...,ENSG00000198899,ENSG00000198938,ENSG00000198840,ENSG00000212907,ENSG00000198886,ENSG00000198786,ENSG00000198695,ENSG00000198727,ENSG00000274847,diagnosis
AAACCCAAGGGCCCTT.21.11,0,0,0,0,0,0,0,0,0,0,...,4,12,4,0,2,4,0,4,0,NCI_with_No_Plaques
AAACCCACAACGGCCT.32.11,0,1,0,0,0,1,2,0,0,2,...,5,5,0,0,0,0,0,3,0,NCI_with_No_Plaques
AAACCCACACAAGCAG.42.2,0,0,0,0,0,1,0,0,0,0,...,58,79,12,5,32,6,0,38,0,NCI_with_No_Plaques
AAACCCAGTGTACGCC.21.11,0,0,0,1,0,0,0,0,0,0,...,18,24,5,0,11,1,0,10,0,NCI_with_No_Plaques
AAACCCATCAGACCCG.6.2,0,0,0,0,0,0,1,0,0,0,...,24,29,13,3,17,5,0,8,0,NCI_with_No_Plaques
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGGTTGTCGCATCG.38.1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,2,0,AD_with_Plaques
TTTGGTTGTGCCTGGT.38.1,0,0,0,0,0,0,0,0,0,0,...,2,5,1,0,2,0,0,0,0,AD_with_Plaques
TTTGGTTTCTGCAGCG.34.2,0,0,0,0,0,0,0,0,0,2,...,3,1,2,0,2,1,0,4,0,AD_with_Plaques
TTTGTCAGTCAGGACA.38.1,0,0,0,0,0,0,0,0,0,0,...,4,23,6,0,9,1,0,8,0,AD_with_Plaques


In [78]:
qdata = qdata.sample(frac = 1)

In [79]:
qdata

ensemble_gene_name,ENSG00000163463,ENSG00000163462,ENSG00000185499,ENSG00000169231,ENSG00000261905,ENSG00000263290,ENSG00000261893,ENSG00000263324,ENSG00000262785,ENSG00000160752,...,ENSG00000198899,ENSG00000198938,ENSG00000198840,ENSG00000212907,ENSG00000198886,ENSG00000198786,ENSG00000198695,ENSG00000198727,ENSG00000274847,diagnosis
AAGCGAGTCGAGCACC.10.2,0,0,0,0,0,0,0,0,0,0,...,28,43,7,2,10,3,0,10,0,AD_with_Plaques
CTTCGGTAGGGCTAAC.10.2,0,0,0,0,0,0,0,0,0,0,...,13,20,3,1,7,3,0,9,0,AD_with_Plaques
AGGACGAGTTTAGTCG.21.11,0,0,0,0,0,0,0,0,0,0,...,20,16,4,0,12,4,0,20,0,NCI_with_No_Plaques
AGATGAAAGGTTTGAA.8.11,0,0,0,0,0,0,0,0,0,1,...,8,14,1,0,3,1,0,7,0,NCI_with_No_Plaques
AAAGTCCTCTCTTGCG.13.11,0,1,0,0,1,0,0,1,0,0,...,38,62,10,0,12,3,0,31,0,NCI_with_No_Plaques
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ATCAGGTTCAGTCACA.10.2,0,0,0,0,0,0,0,0,0,0,...,31,30,3,0,12,3,0,14,0,AD_with_Plaques
ACAGGGAAGGTCGAGT.10.2,0,0,0,0,0,0,0,0,0,0,...,42,44,9,0,15,2,0,17,0,AD_with_Plaques
CGAAGTTTCTGCTTTA.8.11,0,0,0,0,0,0,0,0,0,0,...,1,1,0,1,0,0,0,4,0,NCI_with_No_Plaques
CGTAGGCGTACTTGAC.38.1,0,0,0,0,0,0,0,0,0,0,...,0,10,1,0,2,0,0,2,0,AD_with_Plaques


In [80]:
qlabel = pd.DataFrame(qdata.diagnosis)

In [81]:
qlabel.reset_index(drop = True, inplace = True)
rlabel.reset_index(drop = True, inplace = True)
rlabel

,diagnosis
0,AD_with_Plaques
1,AD_with_Plaques
2,AD_with_Plaques
3,AD_with_Plaques
4,NCI_with_No_Plaques
...,...
48739,AD_with_Plaques
48740,AD_with_Plaques
48741,AD_with_Plaques
48742,NCI_with_No_Plaques


In [82]:
qdata = qdata.drop(columns=['diagnosis'])

In [83]:
qdata

ensemble_gene_name,ENSG00000163463,ENSG00000163462,ENSG00000185499,ENSG00000169231,ENSG00000261905,ENSG00000263290,ENSG00000261893,ENSG00000263324,ENSG00000262785,ENSG00000160752,...,ENSG00000228253,ENSG00000198899,ENSG00000198938,ENSG00000198840,ENSG00000212907,ENSG00000198886,ENSG00000198786,ENSG00000198695,ENSG00000198727,ENSG00000274847
AAGCGAGTCGAGCACC.10.2,0,0,0,0,0,0,0,0,0,0,...,0,28,43,7,2,10,3,0,10,0
CTTCGGTAGGGCTAAC.10.2,0,0,0,0,0,0,0,0,0,0,...,0,13,20,3,1,7,3,0,9,0
AGGACGAGTTTAGTCG.21.11,0,0,0,0,0,0,0,0,0,0,...,0,20,16,4,0,12,4,0,20,0
AGATGAAAGGTTTGAA.8.11,0,0,0,0,0,0,0,0,0,1,...,0,8,14,1,0,3,1,0,7,0
AAAGTCCTCTCTTGCG.13.11,0,1,0,0,1,0,0,1,0,0,...,0,38,62,10,0,12,3,0,31,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ATCAGGTTCAGTCACA.10.2,0,0,0,0,0,0,0,0,0,0,...,0,31,30,3,0,12,3,0,14,0
ACAGGGAAGGTCGAGT.10.2,0,0,0,0,0,0,0,0,0,0,...,0,42,44,9,0,15,2,0,17,0
CGAAGTTTCTGCTTTA.8.11,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,1,0,0,0,4,0
CGTAGGCGTACTTGAC.38.1,0,0,0,0,0,0,0,0,0,0,...,0,0,10,1,0,2,0,0,2,0


In [84]:
rdata = rdata.T
qdata = qdata.T

In [85]:
r_data_tmp = rdata.copy()
q_data_tmp = qdata.copy()
r_label_tmp = rlabel.copy()

In [86]:
import numpy as np
import pandas as pd


def fphi(x):
  zt = (1 - (x + 1).div(x.max(axis=1) + 1, axis=0)).sum(axis=1) / (x.shape[1] - 1)
  zp = (x + 1).div(x.max(axis=1) + 1, axis=0).mul(zt, axis=0)
  return zp


def frho(x):
  zt = (1 - (1 / (x + 1)).mul(x.min(axis=1) + 1, axis=0)).sum(axis=1) / (x.shape[1] - 1)
  zp = (1 / (x + 1)).mul(x.min(axis=1) + 1, axis=0).mul(zt, axis=0)
  return zp


def fmkg(phi, rho, thr1=0.95, thr2=0.9):
  #print(phi)
  #print(rho)
  gnm = phi.index.values
  ctpnm = phi.columns.values
  phi = np.array(phi)
  rho = np.array(rho)
  nummkg1 = round((1 - thr1) * phi.shape[0])
  nummkg2 = round((1 - thr2) * phi.shape[0])
  print(nummkg1)
  print(nummkg2)
  alpha = []
  beta = []
  for i in range(0, phi.shape[1]):
    alpha = np.append(alpha, np.quantile(phi[:, i], thr1))
    beta = np.append(beta, np.quantile(rho[:, i], thr2))
  mkh = []
  mkl = []
  for i in range(0, phi.shape[1]):
    mkh = np.concatenate([mkh, gnm[phi[:, i] >= alpha[i]][0:nummkg1]], axis=0)
    mkl = np.concatenate([mkl, gnm[rho[:, i] >= beta[i]][0:nummkg2]], axis=0)
  
  print(len(mkh))
  print(len(mkl))
  mkh = mkh.reshape(nummkg1, phi.shape[1])
  mkl = mkl.reshape(nummkg2, rho.shape[1])
  mkh = pd.DataFrame(mkh, columns=ctpnm)
  mkl = pd.DataFrame(mkl, columns=ctpnm)
  print(mkh)
  print(mkl)
  return mkh, mkl


def get_expression(rdata, qdata, rlabel, thrh=0.95, thrl=0.9, normalization=True, marker=True):
    # calculate sum of cell type
    rulabel = rlabel.iloc[:, 0].unique()
    rdt = pd.DataFrame(data=None, columns=None)
    for l in rulabel:
      rdata_l = rdata.iloc[:, rlabel[(rlabel["diagnosis"] == l)].index.tolist()] #row value in rlabel, is column in rdata
      zs = rdata_l.apply(lambda x: x.sum(), axis=1)
      rdt = pd.concat([rdt, pd.DataFrame(data=zs, columns=[l])], axis=1)
      

    # normalization
    if normalization:
        rdt_df = rdt
        rdata_df = rdata
        qdata_df = qdata
        rdt = np.array(rdt_df, dtype=np.float32)
        rdata = np.array(rdata_df, dtype=np.float32)
        qdata = np.array(qdata_df, dtype=np.float32)
        rdt = np.divide(rdt, np.sum(rdt, axis=0, keepdims=True)) * 10000
        rdata = np.log2(np.divide(rdata, np.sum(rdata, axis=0, keepdims=True)) * 10000 + 1)
        qdata = np.log2(np.divide(qdata, np.sum(qdata, axis=0, keepdims=True)) * 10000 + 1)

        rdt = pd.DataFrame(data=rdt, columns=rdt_df.columns, index=rdt_df.index)
        rdata = pd.DataFrame(data=rdata, columns=rdata_df.columns, index=rdata_df.index)
        qdata = pd.DataFrame(data=qdata, columns=qdata_df.columns, index=qdata_df.index)
    #print(rdt.head())
    #print(rdata.head())
    # match gene ensembl ids between the query data and the reference data
    nm = qdata.index.tolist()
    zg = pd.DataFrame(data=nm, columns=['gene'])
    gid = []
    for i in range(0, len(nm)):
      if nm[i] not in rdata.index.tolist():
        zg.iloc[i] = ''
      else:
        gid = gid + [nm[i]]
    qdata = qdata.loc[gid, :]
    rdata = rdata.loc[gid, :]
    rdt = rdt.loc[gid, :]

    rlabel.index = rdata.columns

    if not marker:
        train_x = rdata
        train_y = rlabel
        test_x = qdata
    else:
        # calculate important genes
        phi = fphi(rdt)
        rho = frho(rdt)
        mkh, mkl = fmkg(phi, rho, thrh, thrl)
        mkg = np.unique(np.append(np.array(mkh).reshape(1, -1), np.array(mkl).reshape(1, -1)))

        # return training set and testing set
        train_x = rdata.loc[mkg, :]
        train_y = rlabel
        test_x = qdata.loc[mkg, :]
    return train_x, test_x, train_y


In [87]:

r_data_tmp = rdata.copy()
q_data_tmp = qdata.copy()
r_label_tmp = rlabel.copy()
train_x, test_x, train_y = get_expression(r_data_tmp,
                                              q_data_tmp,
                                              r_label_tmp,
                                              thrh=0.99,
                                              thrl=0.99,
                                              normalization=True,
                                              marker=True)

225
225
450
450
     AD_with_Plaques NCI_with_No_Plaques
0    ENSG00000132718     ENSG00000162670
1    ENSG00000230876     ENSG00000143947
2    ENSG00000119866     ENSG00000187605
3    ENSG00000176204     ENSG00000172965
4    ENSG00000175497     ENSG00000170396
..               ...                 ...
220  ENSG00000186732     ENSG00000171189
221  ENSG00000198888     ENSG00000198763
222  ENSG00000198712     ENSG00000198899
223  ENSG00000198840     ENSG00000212907
224  ENSG00000198786     ENSG00000198727

[225 rows x 2 columns]
     AD_with_Plaques NCI_with_No_Plaques
0    ENSG00000132718     ENSG00000162670
1    ENSG00000230876     ENSG00000143947
2    ENSG00000233723     ENSG00000119866
3    ENSG00000187605     ENSG00000176204
4    ENSG00000175497     ENSG00000170396
..               ...                 ...
220  ENSG00000215386     ENSG00000171189
221  ENSG00000198888     ENSG00000198763
222  ENSG00000198712     ENSG00000198899
223  ENSG00000198840     ENSG00000212907
224  ENSG00000198

In [88]:
train_x

,GTCTGTCTCAACTGGT.4.13,CACAAACCAGCTGGCT.19.1,TCCGTGTTCGGTTGTA.27.9,CACTCCACATGTCGAT.28.0,ATTCTTGCACCCATAA.7.6,TACGTCCAGGGTATAT.26.2,AGATGCTTCTCTCTTC.4.13,TCAACGAGTTCCATGA.1.1,AATGGAAGTGACTCGC.13.12,CTGCGAGCATGGGCAA.16.2,...,TCGGTCTTCATGGATC.30.10,TACATTCGTTTGGAAA.7.13,AGTGACTCACGCGCTA.28.2,CTAAGACTCCGCGCAA.23.1,TGACAGTTCTAGTGTG.45.2,GGTGTTATCAACTCTT.31.3,TAGAGTCTCCCATTTA.4.13,GAGTTGTTCTTCTTCC.23.10,AACAAAGGTCGAGTTT.31.9,TTACGTTCATAGAATG.18.8
ensemble_gene_name,,,,,,,,,,,,,,,,,,,,,
ENSG00000006283,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.405885,0.000000,...,0.000000,1.841240,0.000000,0.000000,0.000000,0.000000,2.557091,1.492315,0.000000,0.000000
ENSG00000006459,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.405885,2.171750,...,1.264669,0.000000,0.000000,0.000000,0.000000,0.000000,2.557091,1.492315,0.000000,0.000000
ENSG00000034713,0.000000,0.000000,0.000000,0.000000,0.000000,4.486282,0.000000,1.414887,0.632207,0.907986,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.443756
ENSG00000035862,0.000000,1.840891,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.070299,1.859680,...,0.000000,1.350445,2.940274,0.000000,0.000000,0.000000,2.557091,0.000000,0.000000,0.000000
ENSG00000041515,3.446906,1.196076,3.701834,3.001389,0.000000,0.000000,4.378347,2.115292,3.827658,0.000000,...,0.000000,1.023803,3.843093,0.000000,0.000000,4.091597,0.000000,3.045024,2.410826,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000283510,2.795548,2.284948,1.509993,0.000000,1.030361,4.486282,2.432818,1.414887,1.070299,0.907986,...,1.264669,1.841240,2.940274,0.000000,3.568238,1.768637,2.557091,0.931078,1.659420,1.443756
ENSG00000285390,0.000000,0.000000,0.000000,3.001389,0.000000,0.000000,0.000000,0.000000,0.000000,0.907986,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.849951,0.000000,0.931078,1.659420,1.443756
ENSG00000285479,3.157679,2.898359,2.231486,4.460948,3.699236,0.000000,2.432818,3.222178,3.026648,2.834907,...,4.551118,3.851252,0.000000,5.232099,0.000000,0.849951,2.557091,2.468193,2.410826,2.623129


In [89]:
train_y

,diagnosis
GTCTGTCTCAACTGGT.4.13,AD_with_Plaques
CACAAACCAGCTGGCT.19.1,AD_with_Plaques
TCCGTGTTCGGTTGTA.27.9,AD_with_Plaques
CACTCCACATGTCGAT.28.0,AD_with_Plaques
ATTCTTGCACCCATAA.7.6,NCI_with_No_Plaques
...,...
GGTGTTATCAACTCTT.31.3,AD_with_Plaques
TAGAGTCTCCCATTTA.4.13,AD_with_Plaques
GAGTTGTTCTTCTTCC.23.10,AD_with_Plaques
AACAAAGGTCGAGTTT.31.9,NCI_with_No_Plaques


In [90]:
train_y.diagnosis.value_counts()

diagnosis
AD_with_Plaques        24372
NCI_with_No_Plaques    24372
Name: count, dtype: int64

In [91]:
test_x

,AAGCGAGTCGAGCACC.10.2,CTTCGGTAGGGCTAAC.10.2,AGGACGAGTTTAGTCG.21.11,AGATGAAAGGTTTGAA.8.11,AAAGTCCTCTCTTGCG.13.11,TGGGCGTGTACAGTTC.26.3,CAACAGTAGCAATTAG.13.11,AGATCGTTCCAGGACC.13.2,CATGGTACATTGGGAG.13.2,CTAAGACGTGATGTGG.38.1,...,CAGTTAGGTCATACCA.8.11,AAGCATCCAATCAGCT.21.11,CAATGACCATCATCCC.34.2,CTGTCGTAGAATAACC.21.11,ACACCAAGTGTTCGTA.10.9,ATCAGGTTCAGTCACA.10.2,ACAGGGAAGGTCGAGT.10.2,CGAAGTTTCTGCTTTA.8.11,CGTAGGCGTACTTGAC.38.1,AGCGGTCAGATGTGTA.38.1
ensemble_gene_name,,,,,,,,,,,,,,,,,,,,,
ENSG00000006283,0.000000,0.000000,2.560901,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000006459,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.197652,1.439532,0.000000,...,0.000000,0.000000,0.000000,1.709128,0.000000,0.000000,0.000000,1.812524,0.000000,0.000000
ENSG00000034713,0.000000,2.426632,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.329417,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000035862,0.000000,2.426632,0.000000,2.272885,0.000000,0.000000,0.000000,1.197652,0.000000,0.000000,...,0.000000,1.828763,0.589128,0.000000,1.255002,0.000000,0.000000,0.000000,2.627295,2.652649
ENSG00000041515,0.000000,0.000000,0.000000,0.000000,2.157808,0.000000,4.431697,1.842907,0.000000,2.644341,...,0.000000,3.113862,0.000000,3.626224,4.022671,3.640145,3.414384,0.000000,2.627295,2.652649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000283510,2.197286,0.000000,1.786755,0.000000,0.759133,0.000000,1.144473,0.000000,2.617530,0.000000,...,0.000000,1.828763,0.589128,0.000000,2.367377,0.000000,2.077452,0.000000,0.000000,2.652649
ENSG00000285390,0.000000,0.000000,0.000000,0.000000,0.759133,0.000000,0.000000,0.000000,0.000000,4.066632,...,0.000000,0.000000,1.006244,0.000000,0.000000,0.000000,0.000000,1.812524,0.000000,3.533117
ENSG00000285479,3.316927,0.000000,4.182109,4.798580,3.892338,3.246251,2.818292,2.626356,4.036588,0.000000,...,3.302164,3.113862,3.010909,3.869654,3.676575,1.827177,2.543723,2.590991,0.000000,2.652649


In [101]:
ensembl_pathway_relation = '../../../CellTICS/reactome/Ensembl2Reactome_All_Levels.txt'
species = 'human'
pathway_names = '../../../CellTICS/reactome/ReactomePathways.txt'
pathway_relation = '../../../CellTICS/reactome/ReactomePathwaysRelation.txt'
datatype = 'diagnosis'
n_hidden = 3

In [93]:
pathway_genes = get_gene_pathways(ensembl_pathway_relation, species=species)

In [94]:
#set(pathway_genes.gene)-set(test_x.index)

In [102]:
masking, layers_node, train_x, test_x = get_masking(pathway_names,
                                                        pathway_genes,
                                                        pathway_relation,
                                                        train_x,
                                                        test_x,
                                                        train_y,
                                                        datatype,
                                                        species,
                                                        n_hidden)

In [103]:
train_x

,GTCTGTCTCAACTGGT.4.13,CACAAACCAGCTGGCT.19.1,TCCGTGTTCGGTTGTA.27.9,CACTCCACATGTCGAT.28.0,ATTCTTGCACCCATAA.7.6,TACGTCCAGGGTATAT.26.2,AGATGCTTCTCTCTTC.4.13,TCAACGAGTTCCATGA.1.1,AATGGAAGTGACTCGC.13.12,CTGCGAGCATGGGCAA.16.2,...,TCGGTCTTCATGGATC.30.10,TACATTCGTTTGGAAA.7.13,AGTGACTCACGCGCTA.28.2,CTAAGACTCCGCGCAA.23.1,TGACAGTTCTAGTGTG.45.2,GGTGTTATCAACTCTT.31.3,TAGAGTCTCCCATTTA.4.13,GAGTTGTTCTTCTTCC.23.10,AACAAAGGTCGAGTTT.31.9,TTACGTTCATAGAATG.18.8
ensemble_gene_name,,,,,,,,,,,,,,,,,,,,,
ENSG00000184408,4.234872,4.348185,3.356205,0.000000,3.376126,6.027681,1.677953,3.459213,4.590796,2.428175,...,3.909506,4.660032,4.394151,4.666203,3.952544,1.768637,4.360420,4.907584,3.268371,0.000000
ENSG00000205542,0.000000,3.128777,0.944142,0.000000,0.000000,0.000000,0.000000,1.414887,0.632207,1.859680,...,0.000000,1.350445,0.000000,0.000000,0.000000,0.849951,0.000000,0.000000,0.000000,1.443756
ENSG00000167526,0.000000,1.840891,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.632207,0.000000,...,1.264669,0.600875,2.940274,3.721933,0.000000,0.849951,0.000000,0.000000,0.000000,0.000000
ENSG00000115268,0.000000,1.840891,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.632207,0.000000,...,0.000000,0.600875,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000169946,2.795548,0.000000,0.000000,0.000000,2.859018,0.000000,0.000000,1.414887,1.405885,1.859680,...,1.264669,1.841240,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.443756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000133107,3.446906,3.128777,0.000000,0.000000,0.605274,0.000000,0.000000,2.584762,0.632207,0.000000,...,0.000000,2.359826,0.000000,0.000000,3.042842,0.000000,0.000000,1.895342,0.000000,0.000000
ENSG00000154096,0.000000,2.284948,0.000000,0.000000,0.605274,0.000000,0.000000,0.000000,2.104204,1.859680,...,1.928064,2.206784,2.940274,0.000000,0.000000,0.849951,2.557091,0.931078,0.000000,0.000000
ENSG00000125740,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,2.498180,0.000000,0.000000,0.000000,0.000000,0.000000,0.931078,0.000000,2.150743


In [104]:
test_x

,AAGCGAGTCGAGCACC.10.2,CTTCGGTAGGGCTAAC.10.2,AGGACGAGTTTAGTCG.21.11,AGATGAAAGGTTTGAA.8.11,AAAGTCCTCTCTTGCG.13.11,TGGGCGTGTACAGTTC.26.3,CAACAGTAGCAATTAG.13.11,AGATCGTTCCAGGACC.13.2,CATGGTACATTGGGAG.13.2,CTAAGACGTGATGTGG.38.1,...,CAGTTAGGTCATACCA.8.11,AAGCATCCAATCAGCT.21.11,CAATGACCATCATCCC.34.2,CTGTCGTAGAATAACC.21.11,ACACCAAGTGTTCGTA.10.9,ATCAGGTTCAGTCACA.10.2,ACAGGGAAGGTCGAGT.10.2,CGAAGTTTCTGCTTTA.8.11,CGTAGGCGTACTTGAC.38.1,AGCGGTCAGATGTGTA.38.1
ensemble_gene_name,,,,,,,,,,,,,,,,,,,,,
ENSG00000184408,3.939662,3.820598,2.560901,4.79858,5.037986,0.0000,3.244061,1.842907,0.000000,0.000000,...,3.680165,4.729184,3.596389,5.387272,4.535231,2.269820,4.981661,3.093846,0.000000,3.533117
ENSG00000205542,0.000000,0.000000,0.000000,1.54421,0.000000,2.3908,1.144473,0.000000,2.617530,0.000000,...,1.983549,0.000000,1.329417,0.000000,0.759869,0.000000,0.000000,0.000000,0.000000,2.652649
ENSG00000167526,1.481856,0.000000,1.786755,0.00000,0.000000,0.0000,0.000000,0.000000,1.439532,0.000000,...,0.000000,0.000000,2.179626,0.000000,0.000000,0.000000,0.000000,0.000000,2.627295,2.652649
ENSG00000115268,0.000000,2.426632,0.000000,0.00000,0.000000,2.3908,0.000000,1.197652,0.000000,0.000000,...,0.000000,0.000000,1.329417,0.000000,0.000000,0.000000,0.000000,0.000000,2.627295,0.000000
ENSG00000169946,3.030745,2.426632,3.061973,0.00000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.828763,0.000000,3.333213,0.759869,0.000000,1.384219,0.000000,4.440597,3.533117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000133107,0.000000,0.000000,0.000000,1.54421,1.253956,0.0000,2.211623,0.000000,0.000000,0.000000,...,1.983549,2.609907,1.006244,3.333213,0.759869,0.000000,1.384219,0.000000,0.000000,4.075928
ENSG00000154096,0.000000,2.426632,0.000000,1.54421,1.914523,0.0000,0.000000,2.626356,0.000000,0.000000,...,1.308782,2.609907,1.329417,0.000000,0.000000,1.185365,1.384219,0.000000,0.000000,3.533117
ENSG00000125740,0.000000,0.000000,0.000000,0.00000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [105]:
#pd.DataFrame(test_x.index).to_csv('../../preprocessed_data/inhibitory_neuron/marker_genes.csv', index=False)

In [106]:
masking

{0: array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0]]),
 1: array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [1, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 2: array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 1, 0],
        [0, 0, 1, ..., 0, 0, 0]])}

In [107]:
len(masking)

3

In [112]:
len(layers_node[0])

2

In [108]:
masking[0].shape, len(layers_node[1])

((27, 92), 27)

In [109]:
masking[1].shape, len(layers_node[2])

((92, 216), 92)

In [110]:
masking[2].shape, len(layers_node[3])

((216, 159), 216)

In [116]:
 len(layers_node[4])

159

In [122]:
sum(masking[2].flatten())

716

In [123]:
sum(masking[1].flatten())

216

In [124]:
sum(masking[0].flatten())

93

In [125]:
masking[2].shape

(216, 159)

In [46]:
import numpy as np
import math
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()


def create_placeholders(n_x, n_y):
    X = tf.placeholder(tf.float32, [n_x, None], name="X")
    Y = tf.placeholder(tf.float32, [n_y, None], name="Y")
    return X, Y


def random_mini_batches(X, Y, mini_batch_size=32):
    m = X.shape[1]
    mini_batches = []

    permutation = list(np.random.permutation(m))
    shuffled_X = X[:, permutation]
    shuffled_Y = Y[:, permutation].reshape((Y.shape[0], m))

    num_complete_minibatches = math.floor(m / mini_batch_size)
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[:, k * mini_batch_size: k * mini_batch_size + mini_batch_size]
        mini_batch_Y = shuffled_Y[:, k * mini_batch_size: k * mini_batch_size + mini_batch_size]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[:, num_complete_minibatches * mini_batch_size: m]
        mini_batch_Y = shuffled_Y[:, num_complete_minibatches * mini_batch_size: m]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    return mini_batches


def initialize_parameters(layers_node, n):
    m = len(layers_node)
    W = {}
    b = {}
    M = {}
    for i in range(n - 1):
        j = m - i - 1
        weight = tf.get_variable("W" + str(i + 1), [len(layers_node[j - 1]), len(layers_node[j])],
                                 initializer=tf.keras.initializers.glorot_normal())
        W["W" + str(i + 1)] = weight
        bias = tf.get_variable("b" + str(i + 1), [len(layers_node[j - 1]), 1], initializer=tf.zeros_initializer())
        b["b" + str(i + 1)] = bias
        mask = tf.get_variable("M" + str(i + 1), [len(layers_node[j - 1]), len(layers_node[j])])
        M["M" + str(i + 1)] = mask
    W["W" + str(n)] = tf.get_variable("W" + str(n), [len(layers_node[0]), len(layers_node[m - n])],
                                 initializer=tf.keras.initializers.glorot_normal())
    b["b" + str(n)] = tf.get_variable("b" + str(n), [len(layers_node[0]), 1], initializer=tf.zeros_initializer())
    parameters = {}
    parameters.update(W)
    parameters.update(b)
    parameters.update(M)
    return parameters


def forward_propagation(X, Y, parameters, masking, n, gamma=0.0001):
    W = {}
    b = {}
    M = {}
    for i in range(n):
        W[i + 1] = parameters["W" + str(i + 1)]
        b[i + 1] = parameters["b" + str(i + 1)]
        if i != (n - 1):
            M[i + 1] = parameters["M" + str(i + 1)]
            M[i + 1] = tf.assign(M[i + 1], masking[len(masking) - i - 1])
    Z = {}
    A = {}
    for i in range(1, n + 1):
        if i == 1:
            Z[i] = tf.add(tf.matmul(tf.multiply(M[i], W[i]), X),  b[i])
            A[i] = tf.nn.leakyrelu(Z[i])
        elif i == n:
            Z[i] = tf.add(tf.matmul(W[i], A[i - 1]), b[i])
            A[i] = Z[i]
        else:
            Z[i] = tf.add(tf.matmul(tf.multiply(M[i], W[i]), A[i - 1]),  b[i])
            A[i] = tf.nn.leakyrelu(Z[i])
    logits = tf.transpose(Z[n])
    labels = tf.transpose(Y)
    reg_term = gamma * tf.nn.l2_loss(W[1])
    for i in range(2, n + 1):
        reg_term = reg_term + gamma * tf.nn.l2_loss(W[i])
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels) + reg_term)
    return A, cost


def model(X_train, Y_train, X_test, layers_node, masking, output_layer, learning_rate=0.001, num_epochs=50,
          minibatch_size=32, gamma=0.0001, print_cost=False):
    tf.reset_default_graph()
    (n_x, m) = X_train.shape
    n_y = Y_train.shape[0]
    X, Y = create_placeholders(n_x, n_y)
    parameters = initialize_parameters(layers_node, output_layer)
    A, cost = forward_propagation(X, Y, parameters, masking, output_layer, gamma=gamma)
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    init = tf.global_variables_initializer()

    with tf.Session() as sess:
        sess.run(init)
        for epoch in range(num_epochs):
            epoch_cost = 0.
            num_minibatches = int(m / minibatch_size)
            minibatches = random_mini_batches(X_train, Y_train, minibatch_size)
            for minibatch in minibatches:
                (minibatch_X, minibatch_Y) = minibatch
                _, minibatch_cost = sess.run([optimizer, cost], feed_dict={X: minibatch_X, Y: minibatch_Y})
                epoch_cost += minibatch_cost / num_minibatches
            if print_cost == True:
                print("Cost after epoch %i: %f" % (epoch, epoch_cost))
        output_train = sess.run(A, feed_dict={X: X_train})
        output_test = sess.run(A, feed_dict={X: X_test})
        return output_train, output_test


Instructions for updating:
non-resource variables are not supported in the long term


In [47]:
print_information = True
num_epochs = 100
learning_rate = 0.001
batch_size = 256
l2_regularization = 0.00001
print_cost = True

In [48]:
from neural_network import *

pred_y_df = pd.DataFrame(data=0, index=test_x.columns, columns=list(range(2, len(masking) + 2)))
train_y_df = pd.DataFrame(data=0, index=train_x.columns, columns=list(range(2, len(masking) + 2)))

activation_output = {}
for output_layer in range(2, len(masking) + 2):
    if print_information:
        print("Current sub-neural network has " + str(output_layer - 1) + " hidden layers.")
    output_train, output_test = model(np.array(train_x),
                                          one_hot_coding(train_y, 'diagnosis'),
                                          np.array(test_x),
                                          layers_node,
                                          masking,
                                          output_layer,
                                          num_epochs=num_epochs,
                                          learning_rate=learning_rate,
                                          minibatch_size=batch_size,
                                          gamma=l2_regularization,
                                          print_cost=print_cost)

    train_y_df[output_layer] = get_prediction(output_train[output_layer],
                                                 pd.get_dummies(train_y['diagnosis']),
                                                 train_x,
                                                 datatype='diagnosis')

    
    for j in range(len(output_train)):
        ctp_sort = layers_node[0]
        ctp_sort.sort()
        if j != output_layer - 1:
            output_train[j + 1] = pd.DataFrame(data=output_train[j + 1],
                                                   index=layers_node[len(layers_node) - 2 - j],
                                                   columns=train_x.columns)
        else:
            output_train[j + 1] = pd.DataFrame(data=output_train[j + 1],
                                                   index=ctp_sort,
                                                   columns=train_x.columns)
    activation_output[output_layer] = output_train
    pred_y_df[output_layer] = get_prediction(output_test[output_layer],
                                                 pd.get_dummies(train_y['diagnosis']),
                                                 test_x,
                                                 datatype='diagnosis')
    
pred_test_y = pd.DataFrame(data=0, index=test_x.columns, columns=['diagnosis'])
pred_test_y['diagnosis'] = pred_y_df.T.describe().T['top']

pred_train_y = pd.DataFrame(data=0, index= train_x.columns, columns=['diagnosis'])
pred_train_y['diagnosis'] = train_y_df.T.describe().T['top']

Current sub-neural network has 1 hidden layers.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



2024-07-10 08:33:25.688286: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-10 08:33:25.688456: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-10 08:33:25.688551: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Cost after epoch 0: 0.639706
Cost after epoch 1: 0.564345
Cost after epoch 2: 0.528540
Cost after epoch 3: 0.505899
Cost after epoch 4: 0.491634
Cost after epoch 5: 0.480188
Cost after epoch 6: 0.474730
Cost after epoch 7: 0.468442
Cost after epoch 8: 0.467272
Cost after epoch 9: 0.462905
Cost after epoch 10: 0.456901
Cost after epoch 11: 0.451926
Cost after epoch 12: 0.452375
Cost after epoch 13: 0.446684
Cost after epoch 14: 0.444524
Cost after epoch 15: 0.440945
Cost after epoch 16: 0.437622
Cost after epoch 17: 0.437891
Cost after epoch 18: 0.435486
Cost after epoch 19: 0.431765
Cost after epoch 20: 0.430123
Cost after epoch 21: 0.426767
Cost after epoch 22: 0.424744
Cost after epoch 23: 0.420481
Cost after epoch 24: 0.423648
Cost after epoch 25: 0.418822
Cost after epoch 26: 0.422004
Cost after epoch 27: 0.414492
Cost after epoch 28: 0.411351
Cost after epoch 29: 0.409747
Cost after epoch 30: 0.410856
Cost after epoch 31: 0.405214
Cost after epoch 32: 0.403147
Cost after epoch 33:

/12tb_dsk1/usman/Single_Cell_Microglia_Project/CellTICS/code/utils.py:70: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'AD_with_Plaques' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  y_pred.iloc[i, 0] = train_y.columns.values[output_softmax[:, i].argmax()]
/12tb_dsk1/usman/Single_Cell_Microglia_Project/CellTICS/code/utils.py:70: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'NCI_with_No_Plaques' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  y_pred.iloc[i, 0] = train_y.columns.values[output_softmax[:, i].argmax()]


Current sub-neural network has 2 hidden layers.


...

/12tb_dsk1/usman/Single_Cell_Microglia_Project/CellTICS/code/utils.py:70: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'AD_with_Plaques' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  y_pred.iloc[i, 0] = train_y.columns.values[output_softmax[:, i].argmax()]
/12tb_dsk1/usman/Single_Cell_Microglia_Project/CellTICS/code/utils.py:70: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'NCI_with_No_Plaques' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  y_pred.iloc[i, 0] = train_y.columns.values[output_softmax[:, i].argmax()]


Current sub-neural network has 4 hidden layers.


2024-07-10 08:40:13.400685: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-10 08:40:13.400865: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-10 08:40:13.400961: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Cost after epoch 0: 0.692757
Cost after epoch 1: 0.628618
Cost after epoch 2: 0.588690
Cost after epoch 3: 0.565822
Cost after epoch 4: 0.548349
Cost after epoch 5: 0.539907
Cost after epoch 6: 0.529323
Cost after epoch 7: 0.523014
Cost after epoch 8: 0.517715
Cost after epoch 9: 0.511139
Cost after epoch 10: 0.506722
Cost after epoch 11: 0.504365
Cost after epoch 12: 0.500589
Cost after epoch 13: 0.496892
Cost after epoch 14: 0.495364
Cost after epoch 15: 0.491008
Cost after epoch 16: 0.488972
Cost after epoch 17: 0.487041
Cost after epoch 18: 0.483558
Cost after epoch 19: 0.480762
Cost after epoch 20: 0.477667
Cost after epoch 21: 0.474250
Cost after epoch 22: 0.475901
Cost after epoch 23: 0.469643
Cost after epoch 24: 0.468439
Cost after epoch 25: 0.466358
Cost after epoch 26: 0.464590
Cost after epoch 27: 0.462508
Cost after epoch 28: 0.459619
Cost after epoch 29: 0.457335
Cost after epoch 30: 0.458517
Cost after epoch 31: 0.454462
Cost after epoch 32: 0.451699
Cost after epoch 33:

/12tb_dsk1/usman/Single_Cell_Microglia_Project/CellTICS/code/utils.py:70: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'AD_with_Plaques' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  y_pred.iloc[i, 0] = train_y.columns.values[output_softmax[:, i].argmax()]
/12tb_dsk1/usman/Single_Cell_Microglia_Project/CellTICS/code/utils.py:70: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'NCI_with_No_Plaques' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  y_pred.iloc[i, 0] = train_y.columns.values[output_softmax[:, i].argmax()]


In [53]:
sum(pred_train_y.diagnosis.values == rlabel.diagnosis.values)/len(pred_train_y)

0.936566551780732

In [54]:
sum(pred_test_y.diagnosis.values == qlabel.diagnosis.values)/len(pred_test_y )

0.6165020337013364

In [135]:
output_test[1]

array([[ 1.69132408e-02,  1.69132501e-02,  1.69132482e-02, ...,
         1.69132333e-02,  1.69132277e-02,  1.69132426e-02],
       [ 4.82755604e-06,  4.82755604e-06,  4.82755604e-06, ...,
         4.82755604e-06,  4.82755604e-06,  4.82755604e-06],
       [-9.21799719e-01,  7.57869482e-01,  3.17117929e-01, ...,
         2.44968489e-01,  3.01824361e-01, -2.73294538e-01],
       ...,
       [-4.89185005e-02, -4.89185005e-02, -4.89185005e-02, ...,
        -4.89185005e-02, -4.89185005e-02, -5.74698113e-02],
       [-4.89355111e-03, -4.89355111e-03, -4.89355111e-03, ...,
        -4.89355111e-03, -4.89355111e-03, -4.89355111e-03],
       [-5.32410264e-01,  9.53040868e-02,  6.30982593e-02, ...,
        -4.27476674e-01, -4.75063413e-01,  1.04400806e-01]], dtype=float32)

In [131]:
pred_y_df

,2,3,4,5
GGAGCAAAGTCATTGC.13.11,NCI_with_No_Plaques,NCI_with_No_Plaques,NCI_with_No_Plaques,NCI_with_No_Plaques
ACACGCGAGAAACTGT.21.11,AD_with_Plaques,AD_with_Plaques,NCI_with_No_Plaques,AD_with_Plaques
AACTTCTCAGCTGAGA.21.11,NCI_with_No_Plaques,NCI_with_No_Plaques,NCI_with_No_Plaques,NCI_with_No_Plaques
CTGCGAGGTAGGATAT.6.2,AD_with_Plaques,AD_with_Plaques,NCI_with_No_Plaques,AD_with_Plaques
GCCGATGCATTGAGCT.8.11,NCI_with_No_Plaques,NCI_with_No_Plaques,NCI_with_No_Plaques,NCI_with_No_Plaques
...,...,...,...,...
CATGGTATCCAACACA.13.11,NCI_with_No_Plaques,AD_with_Plaques,NCI_with_No_Plaques,NCI_with_No_Plaques
CTTCCTTAGTGTTCCA.13.2,AD_with_Plaques,NCI_with_No_Plaques,AD_with_Plaques,AD_with_Plaques
GCATGATAGATTGATG.13.11,AD_with_Plaques,NCI_with_No_Plaques,AD_with_Plaques,AD_with_Plaques
ATCCTATGTGACTATC.10.2,NCI_with_No_Plaques,NCI_with_No_Plaques,NCI_with_No_Plaques,AD_with_Plaques


In [51]:
 import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


def create_placeholders(n_x, n_y):
    X = torch.zeros(n_x, 0)
    Y = torch.zeros(n_y, 0)
    return X, Y


def random_mini_batches(X, Y, mini_batch_size=32):
    m = X.shape[1]
    mini_batches = []

    permutation = np.random.permutation(m)
    shuffled_X = X[:, permutation]
    shuffled_Y = Y[:, permutation].reshape((Y.shape[0], m))

    num_complete_minibatches = m // mini_batch_size
    for k in range(num_complete_minibatches):
        mini_batch_X = shuffled_X[:, k * mini_batch_size: (k + 1) * mini_batch_size]
        mini_batch_Y = shuffled_Y[:, k * mini_batch_size: (k + 1) * mini_batch_size]
        mini_batches.append((mini_batch_X, mini_batch_Y))
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[:, num_complete_minibatches * mini_batch_size:]
        mini_batch_Y = shuffled_Y[:, num_complete_minibatches * mini_batch_size:]
        mini_batches.append((mini_batch_X, mini_batch_Y))
    return mini_batches


class CustomNetwork(nn.Module):
    def __init__(self, layers_node, n, masking):
        super(CustomNetwork, self).__init__()
        self.layers = nn.ModuleList()
        self.masks = []
        self.gamma = 0.0001

        for i in range(n - 1):
            in_features = len(layers_node[-i-2])
            out_features = len(layers_node[-i-1])
            self.layers.append(nn.Linear(in_features, out_features))
            self.masks.append(masking[-i-1])

        self.layers.append(nn.Linear(len(layers_node[0]), len(layers_node[-n])))

    def forward(self, x):
        for i, layer in enumerate(self.layers[:-1]):
            mask = self.masks[i]
            x = torch.tanh(layer(x) * mask)
        x = self.layers[-1](x)
        return x

    def compute_cost(self, logits, labels):
        loss = F.cross_entropy(logits, labels)
        reg_term = sum(self.gamma * torch.norm(layer.weight) ** 2 for layer in self.layers)
        cost = loss + reg_term
        return cost


def model(X_train, Y_train, X_test, layers_node, masking, output_layer, learning_rate=0.001, num_epochs=50,
          minibatch_size=32, gamma=0.0001, print_cost=False):
    X_train = torch.tensor(X_train, dtype=torch.float32)
    Y_train = torch.tensor(Y_train, dtype=torch.long)
    X_test = torch.tensor(X_test, dtype=torch.float32)

    (n_x, m) = X_train.shape
    n_y = Y_train.shape[0]

    net = CustomNetwork(layers_node, output_layer, masking)
    optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)  # Using SGD with momentum

    for epoch in range(num_epochs):
        epoch_cost = 0.
        num_minibatches = m // minibatch_size
        minibatches = random_mini_batches(X_train, Y_train, minibatch_size)
        for minibatch in minibatches:
            (minibatch_X, minibatch_Y) = minibatch
            minibatch_X = minibatch_X.permute(1, 0)
            minibatch_Y = minibatch_Y.permute(1, 0)
            optimizer.zero_grad()
            outputs = net(minibatch_X)
            cost = net.compute_cost(outputs, minibatch_Y)
            cost.backward()
            optimizer.step()
            epoch_cost += cost.item() / num_minibatches
        if print_cost:
            print("Cost after epoch %i: %f" % (epoch, epoch_cost))

    net.eval()
    with torch.no_grad():
        output_train = net(X_train.permute(1, 0))
        output_test = net(X_test.permute(1, 0))
    
    return output_train, output_test


In [52]:
qlabel

,diagnosis
0,NCI_with_No_Plaques
1,NCI_with_No_Plaques
2,NCI_with_No_Plaques
3,NCI_with_No_Plaques
4,NCI_with_No_Plaques
...,...
6879,NCI_with_No_Plaques
6880,NCI_with_No_Plaques
6881,NCI_with_No_Plaques
6882,AD_with_Plaques


# Fully Connected